In [ ]:
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
import csv
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time 

# WebDriver ve WebDriverWait başlatılıyor
browser = webdriver.Chrome()
wait = WebDriverWait(browser, 10)

# Kullanıcıdan kitap detaylarını al
kitap_adi = input("Kitap adını giriniz : ")
yayin_evi = input("Yayın evini giriniz : ")

# BKM Kitap için veri çekme fonksiyonu
def bkm_kitap():
    browser.get("https://www.google.com/")
    bkm_kitap_veri_girisi = wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, "textarea.gLFyf")))
    bkm_kitap_veri_girisi.send_keys(f"{kitap_adi} {yayin_evi} site:bkmkitap.com")
    time.sleep(1)
    bkm_kitap_veri_girisi.send_keys(Keys.ENTER)
    
    # İlk arama sonucuna tıklanıyor
    bkm_kitap_tikla = wait.until(EC.element_to_be_clickable((By.XPATH, "//*[@id='rso']/div[1]/div/div/div/div[1]/div/div/span/a")))
    bkm_kitap_tikla.click()  
    time.sleep(1)
    
    # Çerez kabul etme butonu varsa tıklanıyor
    try:
        cerez_kabul_et = wait.until(EC.element_to_be_clickable((By.XPATH, "//*[@id='ccp---nb']/div[1]/div[2]/button[1]")))
        cerez_kabul_et.click()
        time.sleep(1)
    except:
        print("Çerez kabul butonu bulunamadı veya gerekli değil.")

    # Kitap detaylarını sayfadan çekme
    bkm_kitap_soup = BeautifulSoup(browser.page_source, "lxml")
    bkm_kitap_bilgiler = bkm_kitap_soup.find("div", attrs={"id": "product-right"})
    bkm_kitap_adi = bkm_kitap_bilgiler.find("h1").text
    bkm_kitap_yayin_evi = bkm_kitap_bilgiler.find("a").text.strip()
    bkm_kitap_yazar_adi = bkm_kitap_bilgiler.find("a", attrs={"id": "model-title"}).text.strip()
    bkm_kitap_fiyat = bkm_kitap_soup.find("span", attrs={"class": "product-price"}).text

    print("BKM KİTAP = " + bkm_kitap_adi + ", " + bkm_kitap_yayin_evi + ", " + bkm_kitap_yazar_adi + ", " + bkm_kitap_fiyat)

# Kitap Yurdu için veri çekme fonksiyonu
def kitap_yurdu():
    browser.get("https://www.google.com/")
    kitap_yurdu_kitap_veri_girisi = wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, "textarea.gLFyf")))
    kitap_yurdu_kitap_veri_girisi.send_keys(f"{kitap_adi} {yayin_evi} site:kitapyurdu.com")
    time.sleep(1)
    kitap_yurdu_kitap_veri_girisi.send_keys(Keys.ENTER)

    kitap_yurdu_kitap_tikla = browser.find_element(By.XPATH, "//*[@id='rso']/div[1]/div/div/div/div[1]/div/div/span/a")
    kitap_yurdu_kitap_tikla.click()
    time.sleep(3)
    
    # Çerez kabul etme butonu varsa tıklanıyor
    try:
        cerez_kabul_et = wait.until(EC.element_to_be_clickable((By.XPATH, "//*[@id='cookiescript_buttons']")))
        cerez_kabul_et.click()
        time.sleep(1)
    except:
        print("Çerez kabul butonu bulunamadı veya gerekli değil.")

    # Kitap detaylarını sayfadan çekme
    kitap_yurdu_kitap_soup = BeautifulSoup(browser.page_source, "lxml")
    kitap_yurdu_kitap_bilgiler = kitap_yurdu_kitap_soup.find("div", attrs={"class": "pr_details"})
    try:
        kitap_yurdu_kitap_baslik = kitap_yurdu_kitap_bilgiler.find("h1", attrs={"class": "pr_header__heading"}).text.strip()
        kitap_yurdu_kitap_yazar_adi = kitap_yurdu_kitap_bilgiler.find("div", attrs={"class": "pr_producers__item"}).find("a").text.strip()
        kitap_yurdu_kitap_yayin_evi = kitap_yurdu_kitap_bilgiler.find("div", attrs={"class": "pr_producers__publisher"}).find("a").text.strip()
        kitap_yurdu_kitap_fiyat = kitap_yurdu_kitap_bilgiler.find("div", class_="price__item").text.strip()

        print("KİTAP YURDU KİTAP = " + kitap_yurdu_kitap_baslik + ", " + kitap_yurdu_kitap_yayin_evi + ", " + kitap_yurdu_kitap_yazar_adi + ", " + kitap_yurdu_kitap_fiyat + " TL")
    except AttributeError as e:
        print(f"AttributeError: {e}")
        print("Element bulunamadı veya sayfa yapısı değişmiş olabilir.") 

# D&R için veri çekme fonksiyonu
def dr():
    browser.get("https://www.google.com/")
    
    dr_kitap_veri_girisi = wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, "textarea.gLFyf")))
    dr_kitap_veri_girisi.send_keys(f"{kitap_adi} {yayin_evi} site:dr.com.tr")
    time.sleep(1)
    dr_kitap_veri_girisi.send_keys(Keys.ENTER)

    dr_kitap_tikla = browser.find_element(By.XPATH, "//*[@id='rso']/div[1]/div/div/div/div[1]/div/div/span/a")
    dr_kitap_tikla.click()

    # Sayfa kaynağını al ve BeautifulSoup ile ayrıştır
    dr_kitap_soup = BeautifulSoup(browser.page_source, "lxml")

    # Varyantları bulma ve fiyatları çekme
    dr_kitap_variantlar = dr_kitap_soup.find_all("div", class_="variant-item-wrapper")
    fiyat_listesi = []

    for variant in dr_kitap_variantlar:
        # Tüm span öğelerinin metinlerini alıp birleştir
        span_texts = [span.text.strip() for span in variant.find_all("span")]
        span_texts = [text for text in span_texts if text]  # Boş metinleri ayıkla
        if len(span_texts) >= 2:
            variant_tip_fiyat = ": ".join(span_texts[:2])  # İlk iki elemanı (tip ve fiyat) al
        elif len(span_texts) == 1:
            variant_tip_fiyat = span_texts[0]  # Sadece bir metin varsa onu al
        else:
            continue  # Span yoksa, bu varyantı atla
        fiyat_listesi.append(variant_tip_fiyat)

    sonuc = ', '.join(fiyat_listesi)  # Listeyi virgülle birleştir

    # Kitap detaylarını çek
    dr_kitap_bilgiler = dr_kitap_soup.find("div", attrs={"class": "prd-cart-header"})
    dr_kitap_baslik = dr_kitap_bilgiler.find("div", attrs={"class": "prd-name"}).text.strip()
    dr_kitap_yazar_adi = dr_kitap_bilgiler.find("h2", attrs={"class": "author seo-heading"}).find("a").text.strip()
    dr_kitap_yayin_evi = dr_kitap_bilgiler.find("div", attrs={"id": "publisherName"}).find("a").text.strip()

    print("DR KİTAP = " + dr_kitap_baslik + ", " + dr_kitap_yayin_evi + ", " + dr_kitap_yazar_adi + ", " + sonuc)

# Fonksiyonları çalıştır
bkm_kitap()
kitap_yurdu()
dr()
